In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.impute import KNNImputer

In [2]:
# Leer el archivo CSV desde el repositorio de GitHub
url = 'https://raw.githubusercontent.com/cagutig/per-capita-project/refs/heads/main/data/df_gdp_concatenado.csv'
df = pd.read_csv(url)

In [3]:
# Organizar y preparar los datos
df = df.sort_values(['Country Name', 'Year'])
df

,Country Name,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %)
0,Afghanistan,AFG,2000,4.4,0.004828,180.188369,NaN,1.443803
1,Afghanistan,AFG,2001,9.3,0.024169,142.903364,NaN,0.742517
2,Afghanistan,AFG,2002,14.1,1.306950,182.174038,NaN,6.449321
3,Afghanistan,AFG,2003,19.0,1.278493,199.643227,NaN,7.541019
4,Afghanistan,AFG,2004,23.8,3.577104,221.830531,NaN,3.933178
...,...,...,...,...,...,...,...,...
9766,Zimbabwe,ZWE,2019,46.7,1.142806,1421.868596,NaN,1.989253
9767,Zimbabwe,ZWE,2020,52.7,0.699034,1372.696674,NaN,2.031112
9768,Zimbabwe,ZWE,2021,49.0,0.881174,1773.920411,NaN,2.045715
9769,Zimbabwe,ZWE,2022,50.1,1.443729,1676.821489,NaN,2.024036


In [4]:
# Imputar datos faltantes en las columna dadas

# Crear una copia del DataFrame para no modificar el original
df_imputado = df.copy()

# Definir las columnas a imputar
columnas_a_imputar = ['Access to electricity (% of population)', 
                      'Foreign direct investment, net inflows (% of GDP)', 
                      'Government expenditure on education, total (% of GDP)',
                      'Population growth (annual %)']

# Función para imputar columnas seleccionadas
def imputar_knn_columnas_seleccionadas(grupo):
    # Seleccionar las columnas presentes en el grupo
    columnas_presentes = [col for col in columnas_a_imputar if col in grupo.columns]
    
    # Si no hay columnas presentes para imputar, devolver el grupo sin cambios
    if len(columnas_presentes) == 0:
        return grupo

    # Ajustar el número de vecinos dinámicamente según el tamaño del grupo
    n_neighbors = min(3, len(grupo))
    imputer = KNNImputer(n_neighbors=n_neighbors)

    # Verificar si todas las columnas seleccionadas tienen al menos un valor no nulo
    if grupo[columnas_presentes].isnull().all(axis=0).any():
        print(f"Saltando grupo {grupo['Country Name'].iloc[0]}: Todas las columnas están vacías.")
        return grupo
    
    # Imputar solo las columnas presentes
    imputado = pd.DataFrame(
        imputer.fit_transform(grupo[columnas_presentes]),
        columns=columnas_presentes,
        index=grupo.index
    )
    
    # Actualizar únicamente las columnas seleccionadas
    for col in columnas_presentes:
        grupo[col] = imputado[col]
    
    return grupo

# Agrupar por 'Country Name' y aplicar la imputación en cada grupo
df_imputado = df_imputado.groupby('Country Name', group_keys=False).apply(imputar_knn_columnas_seleccionadas)

# Verificar el resultado
df_imputado

Saltando grupo Caribbean small states: Todas las columnas están vacías.
Saltando grupo Cuba: Todas las columnas están vacías.
Saltando grupo Kosovo: Todas las columnas están vacías.
Saltando grupo Montenegro: Todas las columnas están vacías.
Saltando grupo Pacific island small states: Todas las columnas están vacías.


,Country Name,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %)
0,Afghanistan,AFG,2000,4.400000,0.004828,180.188369,4.414443,1.443803
1,Afghanistan,AFG,2001,9.300000,0.024169,142.903364,4.414443,0.742517
2,Afghanistan,AFG,2002,14.100000,1.306950,182.174038,4.414443,6.449321
3,Afghanistan,AFG,2003,19.000000,1.278493,199.643227,4.414443,7.541019
4,Afghanistan,AFG,2004,23.800000,3.577104,221.830531,4.414443,3.933178
...,...,...,...,...,...,...,...,...
9766,Zimbabwe,ZWE,2019,46.700000,1.142806,1421.868596,5.607530,1.989253
9767,Zimbabwe,ZWE,2020,52.700000,0.699034,1372.696674,9.018777,2.031112
9768,Zimbabwe,ZWE,2021,49.000000,0.881174,1773.920411,9.018777,2.045715
9769,Zimbabwe,ZWE,2022,50.100000,1.443729,1676.821489,9.018777,2.024036


In [5]:
# Crear Características con Retardos (Lags)
for col in ['GDP per capita (current US$)', 'Access to electricity (% of population)',
            'Foreign direct investment, net inflows (% of GDP)', 
            'Government expenditure on education, total (% of GDP)',
            'Population growth (annual %)']:
    df_imputado[f'{col}_lag1'] = df_imputado.groupby('Country Name')[col].shift(1)
    df_imputado[f'{col}_lag2'] = df_imputado.groupby('Country Name')[col].shift(2)


In [6]:
df_imputado

,Country Name,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %),GDP per capita (current US$)_lag1,GDP per capita (current US$)_lag2,Access to electricity (% of population)_lag1,Access to electricity (% of population)_lag2,"Foreign direct investment, net inflows (% of GDP)_lag1","Foreign direct investment, net inflows (% of GDP)_lag2","Government expenditure on education, total (% of GDP)_lag1","Government expenditure on education, total (% of GDP)_lag2",Population growth (annual %)_lag1,Population growth (annual %)_lag2
0,Afghanistan,AFG,2000,4.400000,0.004828,180.188369,4.414443,1.443803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2001,9.300000,0.024169,142.903364,4.414443,0.742517,180.188369,NaN,4.4,NaN,0.004828,NaN,4.414443,NaN,1.443803,NaN
2,Afghanistan,AFG,2002,14.100000,1.306950,182.174038,4.414443,6.449321,142.903364,180.188369,9.3,4.4,0.024169,0.004828,4.414443,4.414443,0.742517,1.443803
3,Afghanistan,AFG,2003,19.000000,1.278493,199.643227,4.414443,7.541019,182.174038,142.903364,14.1,9.3,1.306950,0.024169,4.414443,4.414443,6.449321,0.742517
4,Afghanistan,AFG,2004,23.800000,3.577104,221.830531,4.414443,3.933178,199.643227,182.174038,19.0,14.1,1.278493,1.306950,4.414443,4.414443,7.541019,6.449321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,Zimbabwe,ZWE,2019,46.700000,1.142806,1421.868596,5.607530,1.989253,2269.177012,1192.107012,45.4,44.0,2.101721,1.746885,2.050490,5.818780,2.020537,2.043620
9767,Zimbabwe,ZWE,2020,52.700000,0.699034,1372.696674,9.018777,2.031112,1421.868596,2269.177012,46.7,45.4,1.142806,2.101721,5.607530,2.050490,1.989253,2.020537
9768,Zimbabwe,ZWE,2021,49.000000,0.881174,1773.920411,9.018777,2.045715,1372.696674,1421.868596,52.7,46.7,0.699034,1.142806,9.018777,5.607530,2.031112,1.989253
9769,Zimbabwe,ZWE,2022,50.100000,1.443729,1676.821489,9.018777,2.024036,1773.920411,1372.696674,49.0,52.7,0.881174,0.699034,9.018777,9.018777,2.045715,2.031112


In [7]:
# Borrar filas con valores nulos en las columnas lag
columnas_lag = [f'{col}_lag1' for col in columnas_a_imputar] + [f'{col}_lag2' for col in columnas_a_imputar]
df_imputado = df_imputado.dropna(subset=columnas_lag)

# Verificar las dimensiones después de eliminar filas
print(df_imputado.shape)

(9212, 18)


In [8]:
df_imputado

,Country Name,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %),GDP per capita (current US$)_lag1,GDP per capita (current US$)_lag2,Access to electricity (% of population)_lag1,Access to electricity (% of population)_lag2,"Foreign direct investment, net inflows (% of GDP)_lag1","Foreign direct investment, net inflows (% of GDP)_lag2","Government expenditure on education, total (% of GDP)_lag1","Government expenditure on education, total (% of GDP)_lag2",Population growth (annual %)_lag1,Population growth (annual %)_lag2
2,Afghanistan,AFG,2002,14.100000,1.306950,182.174038,4.414443,6.449321,142.903364,180.188369,9.3,4.4,0.024169,0.004828,4.414443,4.414443,0.742517,1.443803
3,Afghanistan,AFG,2003,19.000000,1.278493,199.643227,4.414443,7.541019,182.174038,142.903364,14.1,9.3,1.306950,0.024169,4.414443,4.414443,6.449321,0.742517
4,Afghanistan,AFG,2004,23.800000,3.577104,221.830531,4.414443,3.933178,199.643227,182.174038,19.0,14.1,1.278493,1.306950,4.414443,4.414443,7.541019,6.449321
5,Afghanistan,AFG,2005,28.700000,4.368673,254.115276,4.414443,3.576508,221.830531,199.643227,23.8,19.0,3.577104,1.278493,4.414443,4.414443,3.933178,7.541019
6,Afghanistan,AFG,2006,33.500000,3.413773,274.015392,4.684761,4.139678,254.115276,221.830531,28.7,23.8,4.368673,3.577104,4.414443,4.414443,3.576508,3.933178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,Zimbabwe,ZWE,2019,46.700000,1.142806,1421.868596,5.607530,1.989253,2269.177012,1192.107012,45.4,44.0,2.101721,1.746885,2.050490,5.818780,2.020537,2.043620
9767,Zimbabwe,ZWE,2020,52.700000,0.699034,1372.696674,9.018777,2.031112,1421.868596,2269.177012,46.7,45.4,1.142806,2.101721,5.607530,2.050490,1.989253,2.020537
9768,Zimbabwe,ZWE,2021,49.000000,0.881174,1773.920411,9.018777,2.045715,1372.696674,1421.868596,52.7,46.7,0.699034,1.142806,9.018777,5.607530,2.031112,1.989253
9769,Zimbabwe,ZWE,2022,50.100000,1.443729,1676.821489,9.018777,2.024036,1773.920411,1372.696674,49.0,52.7,0.881174,0.699034,9.018777,9.018777,2.045715,2.031112


In [9]:
# One-Hot Encoding para la Variable de País
df = pd.get_dummies(df_imputado, columns=['Country Name'], drop_first=True)
df

,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %),GDP per capita (current US$)_lag1,GDP per capita (current US$)_lag2,Access to electricity (% of population)_lag1,...,Country Name_Upper middle income,Country Name_Uzbekistan,Country Name_Vanuatu,"Country Name_Venezuela, RB",Country Name_Viet Nam,Country Name_West Bank and Gaza,Country Name_World,"Country Name_Yemen, Rep.",Country Name_Zambia,Country Name_Zimbabwe
2,AFG,2002,14.100000,1.306950,182.174038,4.414443,6.449321,142.903364,180.188369,9.3,...,0,0,0,0,0,0,0,0,0,0
3,AFG,2003,19.000000,1.278493,199.643227,4.414443,7.541019,182.174038,142.903364,14.1,...,0,0,0,0,0,0,0,0,0,0
4,AFG,2004,23.800000,3.577104,221.830531,4.414443,3.933178,199.643227,182.174038,19.0,...,0,0,0,0,0,0,0,0,0,0
5,AFG,2005,28.700000,4.368673,254.115276,4.414443,3.576508,221.830531,199.643227,23.8,...,0,0,0,0,0,0,0,0,0,0
6,AFG,2006,33.500000,3.413773,274.015392,4.684761,4.139678,254.115276,221.830531,28.7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9766,ZWE,2019,46.700000,1.142806,1421.868596,5.607530,1.989253,2269.177012,1192.107012,45.4,...,0,0,0,0,0,0,0,0,0,1
9767,ZWE,2020,52.700000,0.699034,1372.696674,9.018777,2.031112,1421.868596,2269.177012,46.7,...,0,0,0,0,0,0,0,0,0,1
9768,ZWE,2021,49.000000,0.881174,1773.920411,9.018777,2.045715,1372.696674,1421.868596,52.7,...,0,0,0,0,0,0,0,0,0,1
9769,ZWE,2022,50.100000,1.443729,1676.821489,9.018777,2.024036,1773.920411,1372.696674,49.0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
# Dividir el set de datos en entrenamiento y prueba

# Definir la variable dependiente (lo que queremos predecir)
y = df['GDP per capita (current US$)']

# Definir las variables independientes (todas las demás columnas relevantes)
X = df.drop(['GDP per capita (current US$)', 'Country Code'], axis=1)

In [11]:
# División en 80% entrenamiento y 20% validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Modelo RandomForestRegressor

n_estimators = 200
max_depth = 15
min_samples_split = 5
min_samples_leaf = 4

model = RandomForestRegressor(
    n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf,
    random_state=42
)

In [13]:
# Entrenar el modelo ajustado
model.fit(X_train, y_train)

# Realizar predicciones de validación
y_val_pred = model.predict(X_val)
mae = mean_absolute_error(y_val, y_val_pred)
mse = mean_squared_error(y_val, y_val_pred)
rmse = np.sqrt(mse)

In [14]:
# Imprimir las métricas para verificar
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Mean Absolute Error (MAE): 227.32
Mean Squared Error (MSE): 342180.25
Root Mean Squared Error (RMSE): 584.96


In [15]:
# Ver rango del PIB per cápita
min_gdp = df['GDP per capita (current US$)'].min()
max_gdp = df['GDP per capita (current US$)'].max()
mean_gdp = df['GDP per capita (current US$)'].mean()
std_gdp = df['GDP per capita (current US$)'].std()

print(f'''
- Mínimo: ${min_gdp:,.2f}
- Máximo: ${max_gdp:,.2f}
- Promedio: ${mean_gdp:,.2f}
- Desviación Estándar: ${std_gdp:,.2f}
''')


- Mínimo: $0.00
- Máximo: $78,675.48
- Promedio: $2,903.75
- Desviación Estándar: $5,722.59



In [16]:
data_2023 = df_imputado[df_imputado['Year'] == 2023]
data_2024 = data_2023.copy()
data_2024['Year'] = 2024  # Cambiar el año a 2024 para reflejar la predicción
data_2024

,Country Name,Country Code,Year,Access to electricity (% of population),"Foreign direct investment, net inflows (% of GDP)",GDP per capita (current US$),"Government expenditure on education, total (% of GDP)",Population growth (annual %),GDP per capita (current US$)_lag1,GDP per capita (current US$)_lag2,Access to electricity (% of population)_lag1,Access to electricity (% of population)_lag2,"Foreign direct investment, net inflows (% of GDP)_lag1","Foreign direct investment, net inflows (% of GDP)_lag2","Government expenditure on education, total (% of GDP)_lag1","Government expenditure on education, total (% of GDP)_lag2",Population growth (annual %)_lag1,Population growth (annual %)_lag2
86,Africa Eastern and Southern,AFE,2024,31.984441,1.886923,1672.505957,4.077308,2.531587,1642.432039,1545.956697,48.711995,48.100862,1.748597,5.039673,4.628624,4.771326,2.543757,2.607472
150,Africa Western and Central,AFW,2024,50.731506,2.088676,1584.333285,3.070380,2.540864,1788.875347,1769.171853,55.437577,54.224724,1.892716,2.011143,2.905294,3.203484,2.539799,2.573377
190,Albania,ALB,2024,100.000000,7.026168,8367.775731,3.036397,-1.148418,6810.114041,6377.203096,100.000000,100.000000,7.619756,6.796141,2.744330,3.022560,-1.215790,-0.926918
254,Algeria,DZA,2024,99.300000,0.506786,5260.206250,5.610157,1.554019,5023.252932,4216.251285,100.000000,99.800000,0.106408,0.466643,4.749247,5.514033,1.628339,1.657703
285,Angola,AGO,2024,33.833333,-2.501839,2309.521620,2.675852,3.030996,2933.484644,1927.474078,48.500000,48.200000,-6.320564,-6.548542,2.332126,2.297109,3.096753,3.166030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9519,Viet Nam,VNM,2024,99.933333,4.305159,4346.768492,3.016364,0.682173,4179.011790,3759.948329,100.000000,100.000000,4.362406,4.273146,2.887387,2.945175,0.734794,0.844182
9549,West Bank and Gaza,PSE,2024,99.900000,0.595157,3367.606990,4.708957,2.393265,3799.955270,3678.635657,100.000000,100.000000,1.621797,1.951941,5.506923,5.434420,2.425538,2.457039
9613,World,WLD,2024,90.576287,0.746630,13138.327546,4.102621,0.923105,12730.157167,12362.464209,91.352571,91.420332,1.749120,2.324106,3.774872,4.240356,0.793914,0.865486
9706,Zambia,ZMB,2024,25.164821,0.323223,1369.129365,5.157240,2.720528,1456.901570,1134.713454,47.800000,46.700000,-0.223287,1.784078,3.582595,3.106432,2.758032,2.840806


In [17]:
# Generar las características con retardos para 2024
for col in ['GDP per capita (current US$)', 'Access to electricity (% of population)',
            'Foreign direct investment, net inflows (% of GDP)', 
            'Government expenditure on education, total (% of GDP)',
            'Population growth (annual %)']:
    data_2024[f'{col}_lag1'] = data_2023[col]
    data_2024[f'{col}_lag2'] = data_2023[f'{col}_lag1']

# Mantener solo las columnas necesarias
columnas_necesarias = columnas_lag + ['Year']  # Retardos y el año

In [18]:
# Aplicar One-Hot Encoding para la columna 'Country Name'
data_2024 = pd.get_dummies(data_2024, columns=['Country Name'], drop_first=True)

# Asegurar que las columnas coincidan con las del modelo entrenado
missing_cols = set(X_train.columns) - set(data_2024.columns)
for col in missing_cols:
    data_2024[col] = 0  # Añadir columnas faltantes con valores 0

# Ordenar las columnas para que coincidan con las de entrenamiento
data_2024 = data_2024[X_train.columns]

In [19]:
# Realizar las predicciones para 2024
data_2024['GDP per capita (current US$)_predicted'] = model.predict(data_2024)

# Añadir la columna 'Country Name' de nuevo para claridad
data_2024['Country Name'] = data_2023['Country Name'].values

# Mostrar las predicciones por país
predicciones_2024 = data_2024[['Country Name', 'Year', 'GDP per capita (current US$)_predicted']]
predicciones_2024

,Country Name,Year,GDP per capita (current US$)_predicted
86,Africa Eastern and Southern,2024,1654.003705
150,Africa Western and Central,2024,1615.654302
190,Albania,2024,8552.464649
254,Algeria,2024,5597.264862
285,Angola,2024,2323.907976
...,...,...,...
9519,Viet Nam,2024,4609.290916
9549,West Bank and Gaza,2024,3480.656994
9613,World,2024,13389.001420
9706,Zambia,2024,1488.549819
